In [1]:
import sys
sys.path.insert(1, '/Users/Andrew/Documents/PhD/Scripts/Python/pygplates_rev13')
import pygplates
import numpy as np
import pandas as pd
import nvector as nv

In [2]:
rifts_df = pd.read_csv('/Users/Andrew/Documents/PhD/Models/Supercontinent-data/Brune_etal_Supplement_SengorNatalin2001_RiftTable.csv')
ts = pd.read_csv('/Users/Andrew/Documents/PhD/Models/Supercontinent-data/TimescaleHierarchy.csv')

input_rotation_model_410_0 = '/Applications/GPlates-2.0.0/SampleData/FeatureCollections/Rotations/Matthews_etal_GPC_2016_410-0Ma_GK07.rot'
rotation_model = pygplates.RotationModel(input_rotation_model_410_0)
static_polygons_filename = '/Users/Andrew/Documents/PhD/Models/Matthews-410-0Ma/StaticGeometries/StaticPolygons/PresentDay_StaticPlatePolygons_Matthews++.gpml'
ouput_filename = 'output_rifts.gpml'

output_polylines_filename = 'output_polylines.gpml'

In [ ]:
def AgeRangeFromString(age_string):
    #age_string = 'Late Miocene'
    subset = ts.where(ts.Age.str.match(age_string,case=False) | 
                      ts.SubEpoch.str.match(age_string,case=False) | 
                      ts.Epoch.str.match(age_string,case=False) | 
                      ts.Period.str.match(age_string,case=False)).dropna()

    age_range = (np.min(subset.End_Ma),np.max(subset.Start_Ma))
    
    return age_range

amax = []
amin = []

for index,rift in rifts_df.iterrows():
    ar1 = (np.nan,np.nan) # age range start
    ar2 = (np.nan,np.nan) # age range end
    
    if pd.notnull(rift.Old_Age):
        ar1 = AgeRangeFromString(rift.Old_Age)
        if np.isnan(ar1[0]):
            # This is supposed to catch entries whose age name does not match anything
            # in the list
            print 'no match for %s' % rift.Old_Age
            print rift.Region + rift.ID + ': Old_Age ' + rift.Old_Age
        #print rift.Region + rift.ID + ': Old_Age ' + rift.Old_Age + '. ar1[0] ' + str(ar1[0])+ '. ar1[1] ' + str(ar1[1])  
    
    if pd.notnull(rift.Young_Age):
        ar2 = AgeRangeFromString(rift.Young_Age)
        #print rift.Region + rift.ID + ': Young_Age ' + rift.Young_Age + '. ar2[0] ' + str(ar2[0])+ '. ar2[1] ' + str(ar2[1])  
        
    #print (ar1[1],ar2[1])
    tmp1 = np.nanmax([ar1,ar2])
    tmp2 = np.nanmin([ar1,ar2])
    amax.append(tmp1)
    amin.append(tmp2)
    
    #print rift.Old_Age,rift.Young_Age,ar1,ar2,tmp1,tmp2
rifts_df['Old_Age_Ma'] = pd.Series(amax)
rifts_df['Young_Age_Ma'] = pd.Series(amin)


In [4]:
#some rifts have incomplete ages, we drop these here
rifts_na = rifts_df.dropna(subset=['Old_Age_Ma'])

In [33]:
#we convert what spatial data we have to form two lat lon points defining the two end points of our rift
#this is too load it back into gplates as a polyline file
lat1_tmp = []
lon1_tmp = []
lat2_tmp = []
lon2_tmp = []
for index,row in rifts_na.iterrows():
    #print index
    point_latitude = np.nanmean((row.Lat_Min,row.Lat_Max))
    point_longitude = np.nanmean((row.Long_Min,row.Long_Max))
    if point_longitude > row.Long_Min + 50: #this is not a great fix
        print index
        point_longitude = point_longitude + 180  #this is definitely not a great fix
    orientation = row.Orientation
    rift_length = row.Length
#   rift_age = np.nanmean((row.Old_Age_Ma,row.Young_Age_Ma))
    if type(orientation)==str:
        tmp = []
        for o in orientation.split(','):
            if type(o)==float:
                tmp.append(o)
            elif o=='N':
                tmp.append(0.)
            elif o=='NNE':
                tmp.append(22.5)
            elif o=='NE':
                tmp.append(45.)
            elif o=='ENE':
                tmp.append(67.5)
            elif o=='E':
                tmp.append(90.)
            elif o=='ESE':
                tmp.append(112.5)
            elif o=='SE':
                tmp.append(135.)
            elif o=='SSE':
                tmp.append(157.5)
            elif o=='S':
                tmp.append(180.)
            elif o=='SSW':
                tmp.append(202.5)
            elif o=='SW':
                tmp.append(225.)
            elif o=='WSW':
                tmp.append(247.5)
            elif o=='W':
                tmp.append(270.)
            elif o=='WNW':
                tmp.append(292.5)
            elif o=='NW':
                tmp.append(315.)
            elif o=='NNW':
                tmp.append(337.5)
            else:
                tmp.append(float(o))
    x = y = 0
    for angle in tmp:
        x += np.cos(np.radians(angle))
        y += np.sin(np.radians(angle))
    average_angle = np.degrees(np.arctan2(y, x))
      
    frame = nv.FrameE(a=6371, f=0)
    pointA = frame.GeoPoint(latitude=point_latitude, longitude=point_longitude, degrees=True)
    pointB, _azimuthb = pointA.geo_point(distance=float(rift_length)/2., azimuth=average_angle,degrees=True)
    lat1, lon1 = pointB.latitude_deg, pointB.longitude_deg
    #print rift_length
    #print lat,lon,point_latitude,point_longitude
    pointB, _azimuthb = pointA.geo_point(distance=float(rift_length)/2., azimuth=average_angle+180.,degrees=True)
    lat2, lon2 = pointB.latitude_deg, pointB.longitude_deg
    lat1_tmp.append(lat1)
    lon1_tmp.append(lon1)
    lat2_tmp.append(lat2)
    lon2_tmp.append(lon2)
rifts_na['lat1'] = lat1_tmp
rifts_na['lon1'] = lon1_tmp
rifts_na['lat2'] = lat2_tmp
rifts_na['lon2'] = lon2_tmp

205


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-

In [32]:
rifts_na[200:210]

,Region,ID,Long_Min,Long_Max,Lat_Min,Lat_Max,Orientation,Name,Length,Width,...,Kinematic_Type_1,Geometric_Type_1,Kinematic_Type_2,Geometric_Type_2,Old_Age_Ma,Young_Age_Ma,lat1,lon1,lat2,lon2
205,EEUR,67,-170.0,150.0,74.0,79.0,"285,315",North Chukchi,1300,300.0,...,k22,g3,NaN,NaN,100.5,23.030,78.293992,144.231728,72.828556,-172.616985
206,EEUR,68,172.0,176.0,71.0,75.0,25,Vikitsky,440,70.0,...,k22,g3,NaN,NaN,100.5,23.030,74.770842,177.184167,71.188821,171.406507
207,EEUR,69,165.0,175.0,67.0,70.0,295,Hope,900,240.0,...,k31,g4,NaN,NaN,100.5,23.030,69.889093,159.279260,66.515288,179.236162
208,EEUR,70,130.0,137.0,47.0,50.0,45,Middle Amur,240,120.0,...,k32,g4,NaN,NaN,100.5,5.333,49.257245,134.669220,47.731264,132.365456
211,EEUR,72,79.5,82.5,15.0,17.0,NE,Krishna-Godavari,350,80.0,...,k1,g2,NaN,NaN,145.0,100.500,17.109667,82.164319,14.884134,79.848578
212,EEUR,73,86.5,NaN,20.5,NaN,ENE,Mahanadi,200,100.0,...,k1,g2,NaN,NaN,145.0,100.500,20.841886,87.389037,20.153609,85.614946
213,EEUR,74,87.0,89.0,20.0,24.0,NNE,West Bengal,650,150.0,...,k1,g2,NaN,NaN,145.0,100.500,24.695636,89.230663,19.295526,86.815355
214,EEUR,75,88.0,92.0,22.0,25.0,NNE,Faridpur,350,125.0,...,k1,g2,NaN,NaN,145.0,100.500,24.952594,90.664209,22.044651,89.350292
218,EEUR,79,128.0,130.0,49.0,52.0,"30,80,50",Amursk,250,150.0,...,k41,NaN,k31,NaN,163.5,100.500,51.164302,130.435915,49.818529,127.604422
219,EEUR,80,120.0,127.0,42.0,49.0,35,Songliao,850,340.0,...,k41,NaN,k31,NaN,163.5,100.500,48.584871,126.813438,42.329378,120.535479


In [34]:
# A function that create a polyline feature from some points and adds to a list of features.
def add_polyline_feature_from_points(polyline_features, points, line_number):

    # If have no points then nothing to do.
    if not points:
        return

    # Need at least two points for a polyline.
    if len(points) >= 2:
        polyline = pygplates.PolylineOnSphere(points)

        polyline_feature = pygplates.Feature() # 'unclassified' feature
        polyline_feature.set_geometry(polyline)

        polyline_features.append(polyline_feature)

    # If only one point then emit warning.
    else:
        print 'Line %d: Ignoring polyline - polyline has only one point.' % (line_number-1)

    # Clear points list for next feature.
    # Clear the list in-place so that all references to the list see an empty list.
    del points[:]

In [35]:
#this is where we actually convert to polyline
polyline_features = []
polyline_points1 = []
polyline_points2 = []
for index,row in rifts_na.iterrows():
    line_number = index
    #print row.lat1
    polyline_points1.append(pygplates.PointOnSphere(row.lat1, row.lon1))
    polyline_points1.append(pygplates.PointOnSphere(row.lat2, row.lon2))

    add_polyline_feature_from_points(polyline_features, polyline_points1, line_number)
    
#we also need begin and end ages in order for it to display
beginage = []
endage = []
names = []
#loop through and extract the valid ages
for index, rows in rifts_na.iterrows():
    #print rows.Old_Age_Ma
    beginage.append(rows.Old_Age_Ma)
    #print beginage
    endage.append(rows.Young_Age_Ma)
    names.append(rows.Name)

#zip them together
ages = zip(beginage, endage)

#assign valid times using pygplates
for i,j in zip(polyline_features, ages):
    i.set_valid_time(j[0], j[1])
    
for i,j in zip(polyline_features, names):
    i.set_name(j)

In [36]:
#now we assign our rifts a plate ID based on their present day position
assigned_polyline_features = pygplates.partition_into_plates(
    static_polygons_filename,
    rotation_model,
    polyline_features,
    properties_to_copy = [pygplates.PartitionProperty.reconstruction_plate_id])

#so now our original data points are polylines, have a begin/end time and a plate ID so we can reconstruct them
#these lines save the output
assigned_polyline_feature_collection = pygplates.FeatureCollection(assigned_polyline_features)
assigned_polyline_feature_collection.write(output_polylines_filename)